In [1]:
import json
import requests
import tarfile
import zipfile
import getpass
import os
from tqdm import tqdm

In [2]:
pathOut = "~/data/polar/source/"
pathOut = os.path.expanduser(pathOut)

if not os.path.exists(pathOut):
    os.makedirs(pathOut)

In [3]:
with open('../cube.geojson', 'r') as f:
    cube_specs = json.load(f)

In [4]:
cube_specs['properties']['sources']

[{'name': 'Repeat Subglacial Lake Drainage and Filling Thwaites Glacier',
  'variable_names': ['Ice Sheets', 'Ice Shelves'],
  'download_url': 'https://4d-antarctica.org/wp-content/uploads/2021/01/Malczyk_etal_2020_data_v2.tar',
  'protocol': 'https',
  'credentials': False,
  'compressed': True,
  'compression_format': 'tar',
  'source_format': 'tiff',
  'source_crs': None},
 {'name': 'Supra-glacial lakes West Antarctic Ice Sheet 2017-01',
  'variable_names': ['Ice Sheets', 'Ice Shelves'],
  'download_url': 'https://zenodo.org/record/5642755/files/WAIS_Max_Extent.zip',
  'protocol': 'https',
  'credentials': False,
  'compressed': True,
  'compression_format': 'zip',
  'source_format': 'shp',
  'source_crs': 'EPSG:3031'},
 {'name': 'Curie depth estimates 200x200km xyz data',
  'variable_names': ['Magnetic proxy for GHF'],
  'download_url': 'https://download.pangaea.de/dataset/932452/files/200km_Dziadek_etal_CurieDepth_Results.nc',
  'protocol': 'https',
  'credentials': False,
  'comp

In [5]:
def download_file(source):
    output_source_path = pathOut + "/" + source['name'].lower().replace(" ","_")
    if not os.path.exists(output_source_path):
        os.makedirs(output_source_path)
    url = source['download_url']
    filename = url.split("/")[-1]
    if source['credentials']:
        user = getpass.getpass("username:")
        password = getpass.getpass("password")
        response = requests.get(url,auth=(user,password))
    else:
        response = requests.get(url)
    output = output_source_path + "/" + filename
    open(output, "wb").write(response.content)
    if source['compressed']:
        if source['compression_format'] == 'tar':
            tar = tarfile.open(output, "r:")
            tar.extractall()
            tar.close()
        elif source['compression_format'] == 'zip':
            with zipfile.ZipFile(output,"r") as zip_ref:
                zip_ref.extractall()

In [6]:
[download_file(source) for source in tqdm(cube_specs['properties']['sources'])]

100%|██████████| 9/9 [01:55<00:00, 12.85s/it]


[None, None, None, None, None, None, None, None, None]

In [14]:
with zipfile.ZipFile("/home/davemlz/data/polar/source/supra-glacial_lakes_west_antarctic_ice_sheet_2017-01/WAIS_Max_Extent.zip","r") as zip_ref:
    zip_ref.extractall()

In [11]:
pathOut

'/home/davemlz/data/polar/source/'

In [12]:
tar = tarfile.open("/home/davemlz/data/polar/source/repeat_subglacial_lake_drainage_and_filling_thwaites_glacier/Malczyk_etal_2020_data_v2.tar", "r:")
tar.extractall()
tar.close()

In [15]:
import xarray as xr
import rioxarray

In [22]:
da = rioxarray.open_rasterio("/home/davemlz/data/polar/source/repeat_subglacial_lake_drainage_and_filling_thwaites_glacier/Thw142_2013_mask.tif")

In [25]:
da[0]

<xarray.DataArray (y: 835, x: 1401)>
[1169835 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 -1.82e+06 -1.819e+06 ... -1.12e+06 -1.12e+06
  * y            (y) float64 -2.492e+05 -2.497e+05 ... -6.657e+05 -6.662e+05
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    Band_1:         Band 1
    scale_factor:   1.0
    add_offset:     0.0
    long_name:      Band 1

In [13]:
import geopandas as gpd

In [14]:
gdf = gpd.read_file("WAIS_Max_Extent/WAIS_Jan_2017_Polygons.shp")

In [22]:
gdf.crs.to_epsg()

3031

In [75]:
import xarray as xr

In [78]:
xr.open_dataset("300km_GHF_windowCenters.nc")

<xarray.Dataset>
Dimensions:    (side: 2, xysize: 11526801)
Dimensions without coordinates: side, xysize
Data variables:
    x_range    (side) float64 ...
    y_range    (side) float64 ...
    z_range    (side) float64 ...
    spacing    (side) float64 ...
    dimension  (side) int32 ...
    z          (xysize) float32 ...
Attributes:
    title:    
    source:

In [60]:
from requests.auth import HTTPDigestAuth

requests.get(cube_specs['properties']['sources'][2]['download_url'],auth=HTTPDigestAuth("dml.mont@gmail.com","D$1144070117m"))

<Response [401]>

In [41]:
download_file(cube_specs['properties']['sources'][2])

BadZipFile: File is not a zip file

In [19]:
response = 

In [ ]:
o